# Utilisation de l'API Datagrosyst (Python)

Le but de ce code est d'utiliser l'API de Datagrosyst afin de générer une requète de téléchargement des tables que vous indiquerez au préalable. Celui-ci va permettre de recevoir le mail avec le lien Filesender de téléchargement. L'étape de téléchargement via le filesender n'est pas automatiser, il faudra aller manuellement dans votre boite et mail, cliquer sur le lien et télécharger.

Nous allons également vous faire une code qui ira chercher dans votre dossier de téléchargement le fichier Filesender, pour le décompréssé et le stocker dans un autre dossier (votre dossier de travail).

Ensuite, un dernier bout de code afin d'importer les tables que vous venez de télécharger.

## Imports des librairies

In [ ]:
import os, tarfile, requests, sys
import pandas as pd
from tqdm import tqdm
from datetime import datetime, date

## Initialisation de la requète et des chemins de dossier

<font color="#900000">Attention</font>  aux noms des tables !  
Les chaînes de caractères permettant d’identifier les tables dans le champ **"tables"** sont celles qui sont disponibles dans le volet à droite sur Datagrosyst. Celles-ci diffèrent parfois du nom disponible sur l’interface de sélection. Par exemple, dans le magasin_can, lorsqu’on sélectionne *“domaine”*, on voit apparaître à droite *“domaine_magasin_can”*. C’est ce dernier libellé qu’il faut saisir dans le champ **"tables"** pour obtenir table domaine du magasin_can, car saisir simplement *“domaine”* ferait référence à la table domaine de l’entrepôt.


<font color="#900000">Attention</font> aux restrictions d'accès aux données !  
Si vous saisissez dans **“tables”** des noms de tables pour lesquelles vous n’avez pas les droits, celles-ci ne seront pas ajoutées dans l’export obtenu.

Le <font color="green">token de l'API</font> est en quelque sorte un mot de passe, ne pas le transmettre !  
Vous le trouverez dans [🔗 votre profil](https://agrosyst.fr/datagrosyst/profil) sur Datagrosyst, au dessus des paramètres.

In [ ]:
url = 'https://agrosyst.fr/datagrosyst/export'

# La liste des tables à récupérer
tables = ["domaine",'parcelle'] # A modifier

# La strucuture de la requète
# Modifier "mail" et "token" avec vos informations personnelles disponibles dans le profil de votre compte Datagrosyst
data_request = {
    "mail": "votreadressemail@exemple.fr",      # A modifier
    "token": "tokendiponiblesurDatagrosyst",    # A modifier
    "tables": tables           
}

# Les chemins vers vos dossiers locaux
path_download = "/chemin/vers/ton/dossier/de/Téléchargements"       # Modifier ici le chemin vers ton dossier de téléchargements
path_workdirectory = "/chemin/vers/ton/Workdirectory"               # Modifier ici le chemin vers ton dossier de travail

## Envoyé la demande à Datagrosyst grace à `requests`

In [ ]:
# Envoi de la requête POST à l'API Datagrosyst
response = requests.post(url, json=data_request)

# Code de vérification de la réponse de l'API
if response.status_code == 200:
    print("Requête réussie !")
else:
    print(f"Erreur lors de la requête. Code d'état : {response.status_code}")
    print("Réponse :", response.text)

In [ ]:
sys.exit("Vous avez terminé la phase de demande des données via API, Félicitations !\nPour continuer, il vous faut désormais faire l'étape manuelle de téléchargement des données (lien Filesender dans votre boite mail ; voir 'Télécharger les données').")

## Télécharger les données

**<font color="#900000">Attention cette étape est manuelle !</font>**  
Vous aller recevoir un mail de la part de Filesender vous invitant à aller télécharger les données stockées pour vous via un lien. Cliquez sur ce lien, puis télécharger le fichier compréssé. Ce fichier va noramlement se retrouver directement dans votre dossier de téléchargement **path_download**.

## Déplacer et décompréssé le fichier Filesender téléchargé

Vous avez désormais le fichier compréssé dans votre dossier de téléchargement. Avec le code proposé ci-dessous, vous aller en plusieurs étapes : 
- **Trouver le fichier .tar.gz** (extension des fichiers compréssés) *le plus récent* de votre dossier de téléchargement
- **Copier** ce fichier vers votre dossier de travail
- **Décompréssé** le fichier, ce qui va automatiquement écraser les anciennes tables pour mettre les nouvelles tables (celles présentes dans le fichier compréssé, que vous avez défini lors de l'initialisation)
- **Supprimé** le fichier compréssé pour ne gardé que les tables

In [ ]:
# Trouver le fichier .tar.gz le plus récent du dossier de téléchargements
tar_files = [f for f in os.listdir(path_download) if f.endswith('.tar.gz')]
if not tar_files:
    raise FileNotFoundError("Aucun fichier .tar.gz trouvé.")

latest_file = max(
    [os.path.join(path_download, f) for f in tar_files],
    key=os.path.getmtime
)
file_mtime = datetime.fromtimestamp(os.path.getmtime(latest_file)).date()  # Date de modification

if file_mtime != date.today():
    raise SystemExit(f"Le fichier le plus récent trouvé n'a pas été modifié aujourd'hui.")

print(f"Fichier le plus récent : {latest_file}")

# Copier le fichier compréssé vers le dossier de travail
os.makedirs(path_workdirectory, exist_ok=True)
dest_tar_path = os.path.join(path_workdirectory, os.path.basename(latest_file))

with open(latest_file, 'rb') as src, open(dest_tar_path, 'wb') as dst:
    dst.write(src.read())

# Décompresser (écrase automatiquement les fichiers existants, càd les plus anciens)
with tarfile.open(dest_tar_path, "r:gz") as tar:
    tar.extractall(path=path_workdirectory, filter="data")

# Supprime le fichier compressé dans le dossier de travail après décompression
os.remove(dest_tar_path) 

# Fin
print(f"Décompression terminée dans {path_workdirectory}.")

## Chargement des données dans le Notebook

Vous voici avec les tables de données nouvellement acquise dans votre dossier de travail !  
Maintenant nous allons les importer dans le Notebook. Pour cela nous allons définir un dictionnaire de table dont le nom sera le nom du fichier. Nous préparons aussi une fonction d'import d'une table qui sera reprise par la fonction d'import de plusieurs tables. Nous utilisons ensuite cette dernière fonction en spécifiant les **tables** que vous venez de téléchargées (pas toutes celle du dossier de travail !) et le chemin vers le dossier de travail.

In [ ]:
# Création du dictionnaire des dataframes
df = {}

# Définition des fonctions d'importation
def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+'/'+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False).replace({'\r\n': '\n'}, regex=True)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

# Import des données que vous venez de télécharger et placer dans le dossier de travail
import_dfs(tables, path_workdirectory, sep = ',', verbose=False)
# df est un dictionnaire contenant les tables domaines et parcelles

# Voir un résumé du contenu du dictionnaire des premiers dataframes
for i, (name, dataframe) in enumerate(df.items()):
    if i < 5:  # Limite aux 5 premières tables
        print(f"\n--- {name} ---")
        print(f"Lignes: {dataframe.shape[0]}, Colonnes: {dataframe.shape[1]}","\n")
        print(dataframe.columns.tolist(), '\n')

# Fin